# Create dataset for wrong assignment bugs

Using the commits

In [ ]:
import os
from pathlib import Path
import codecs
import json
from typing import List, Dict, Any
import pandas as pd
from multiprocessing import Pool, cpu_count
from tqdm.notebook import trange, tqdm

benchmarks_dir = '../../../benchmarks'

real_bugs_dataset_file_path = os.path.join(benchmarks_dir, 'assignments_real_bugs.pkl')
real_bugs_dataset_dir = os.path.join(benchmarks_dir, 'assignments_real_bugs')

In [ ]:
def read_json_file(json_file_path) -> Dict:
    try:
        obj_text = codecs.open(json_file_path, 'r', encoding='utf-8').read()
        return json.loads(obj_text)
    except FileNotFoundError:
        print(
            "Please provide a correct file p. Eg. ./results/validated-conflicts.json")
        return {}
    except Exception as e:
        # Empty JSON file most likely due to abrupt killing of the process while writing
        # print (e)
        return {}


def read_dataset_given_files(extracted_data_files: List) -> pd.DataFrame:
    d = []
    with Pool(cpu_count()) as p:
        with tqdm(total=len(extracted_data_files)) as pbar:
            pbar.set_description_str(
                desc="Reading dataset from files", refresh=False)
            for i, each_vars in enumerate(
                    p.imap_unordered(read_json_file, extracted_data_files, 20)):
                pbar.update()
                d.extend(each_vars)
            p.close()
            p.join()
    extracted_dataset = pd.DataFrame(d)
    return extracted_dataset


def file_path_to_dataset(dataset_file_path, dir_path):
    if not Path(dataset_file_path).is_file():
        file_paths = list(Path(dir_path).rglob('*.json'))
        print(f"Number of files={len(file_paths)}")
        dataset = read_dataset_given_files(extracted_data_files=file_paths)
        print(f"Saving {dataset_file_path}")
        dataset.to_pickle(dataset_file_path, 'gzip')
    else:
        print(f'Reading from {dataset_file_path}')
        dataset = pd.read_pickle(dataset_file_path, 'gzip')
    print(f"Dataset contains {len(dataset)} examples")
    return dataset

In [ ]:
def get_file_loc(row):
    d = row.to_dict()
    if 'benchmarks/real_bugs_github/buggy_' in d['src']:
        file_name = d['src'].replace('benchmarks/real_bugs_github/buggy_', '')
    else:
        file_name = d['src'].replace('benchmarks/real_bugs_github/correct_', '')
    range = str(d['range'][0])
    return file_name + '_' + range

In [ ]:
dataset = file_path_to_dataset(dataset_file_path=real_bugs_dataset_file_path, dir_path=real_bugs_dataset_dir)
row_iter = [row for _, row in dataset.iterrows()]
locations = []
for row in tqdm(row_iter):
    loc = get_file_loc(row)
    locations.append(loc)
dataset['filename_loc'] = locations

In [ ]:
dataset['filename_loc']

In [ ]:
correct_dataset = dataset[dataset['src'].apply(lambda x: 'correct_' in x)]
buggy_dataset = dataset[dataset['src'].apply(lambda x: 'buggy_' in x)]

In [ ]:
correct_dataset.iloc[0, 8]

In [ ]:
buggy_dataset.iloc[0, 8]

In [ ]:
print(f'Length of correct dataset {len(correct_dataset)}')
print(f'Length of buggy dataset {len(buggy_dataset)}')

In [ ]:
merged = correct_dataset.merge(buggy_dataset, left_on='filename_loc', right_on='filename_loc',
                               suffixes=['_CORRECT', '_BUGGY'])
merged

In [ ]:
def get_buggy_non_buggy_data(row):
    d = row.to_dict()
    correct = {k.replace('_CORRECT', ''): v for k, v in d.items() if '_CORRECT' in k}
    correct['probability_that_incorrect'] = 0
    buggy = {k.replace('_BUGGY', ''): v for k, v in d.items() if '_BUGGY' in k}
    buggy['probability_that_incorrect'] = 1
    if correct['lhs'] == buggy['lhs'] and correct['rhs'] != buggy['rhs']:
        return [correct, buggy]
    else:
        return []

In [ ]:
correct_assgn = []
buggy_assgn = []
x_y_pair_given = []
for _, row in tqdm(list(merged.iterrows()), desc='Get lines'):
    r = get_buggy_non_buggy_data(row)
    if len(r):
        correct_assgn.append(r[0])
        buggy_assgn.append(r[1])
        x_y_pair_given.append(r)
print(f'Number of buggy/correct assignments extracted are {len(correct_assgn)}')

In [ ]:
def write_json(content, out_file):
    with open(out_file, 'w+') as f:
        print(f'Writing to {f.name}')
        json.dump(content, f)


write_json(x_y_pair_given, os.path.join(benchmarks_dir, 'correct_buggy_real_wrong_assignments.json'))